In [1]:
from src.fistanet.M5FISTANet import FISTANet
from src.fistanet.M5FISTANetNoST import FISTANetNoST
from src.fistanet.loader import DataSplit
from src.fistanet.solver import Solver
from os.path import join as pjoin
from torchsummary import summary
from datetime import datetime
import numpy as np
import torch

In [2]:
DATA_DIR = './data'
DATA_FILE_GEN = 'generated/BW_master_10000_2024-04-07-12-43-32.pkl'
DATA_FILE_SIGS = 'steinbrinker/testing_data_mvg_avg.npy'
DATA_FILE_BW = 'mit-bih/bw'
DATA_FILE_GAUSS = 'generated/gaussian_noise.npy'
# DATA_FILE_BPDN = 'generated/BW_alphas-BPDN_10000_2024-04-07-12-43-32.npy'
# DATA_FILE_BPDN = 'generated/BW_alphas-BPDN-1iters_10000_2024-04-07-12-43-32.npy'
# DATA_FILE_BPDN = 'generated/BW_alphas-BPDN-3iters_10000_2024-04-07-12-43-32.npy'
DATA_FILE_BPDN = 'generated/BW_alphas-BPDN-5iters_10000_2024-04-07-12-43-32.npy'
DICT_FILE_BW = 'steinbrinker/dictionary_BW_real_data.npy'
NOISE_TYPE = 'bw'
if NOISE_TYPE == 'bw':
    DATA_FILE_NOISE = DATA_FILE_BW
elif NOISE_TYPE == 'gauss':
    DATA_FILE_NOISE = DATA_FILE_GAUSS
DATA_SIZE = 10000
BATCH_SIZE = 1000
TVT_SPLIT = {
    'train': 80,
    'valid': 10,
    'test': 10
}

FNET_LAYER_NO = 4
FNET_FEATURE_NO = 32
LAMBDA_SP_LOSS = 1
LAMBDA_PRED_SP_LOSS = 0
LAMBDA_SYM_LOSS = 1e-1

EPOCH_NO = 10000
TEST_EPOCH = 10001
LR_DEC_AFTER = 10000
LR_DEC_EVERY = 10
START_EPOCH = 0
START_RUN = '2024-05-05-13-10-59'
LOG_INTERVAL = 4
LEARNING_RATE = 1e-3



# DATA_FILE_GEN = 'generated/BW_master_7999-8000_2024-04-07-12-43-32.pkl'
# DATA_SIZE = 2
# BATCH_SIZE = 1
# TVT_SPLIT = {
#     'train': 50,
#     'valid': 50,
#     'test': 0
# }
# FNET_LAYER_NO = 4
# FNET_FEATURE_NO = 16
# LEARNING_RATE = 1e-3
# LAMBDA_SP_LOSS = 0.5

In [3]:
trn_ldr, val_ldr, tst_ldr = DataSplit(DATA_DIR, NOISE_TYPE, DATA_FILE_GEN, DATA_FILE_SIGS, DATA_FILE_NOISE, DATA_FILE_BPDN, TVT_SPLIT, BATCH_SIZE)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
Psi = np.load(pjoin(DATA_DIR, DICT_FILE_BW))
Psi = torch.from_numpy(Psi)
Psi = Psi.clone().detach().to(device=device)

In [6]:
fista_net = FISTANet(FNET_LAYER_NO, FNET_FEATURE_NO)
fista_net = fista_net.to(device)# define arguments of fista_net


In [7]:
# fista_net = FISTANetNoST(FNET_LAYER_NO, FNET_FEATURE_NO)
# fista_net = fista_net.to(device)# define arguments of fista_net


In [8]:
# summary(fista_net, input_size=(1, 64, 298), device=str(device))
print('Total number of parameters fista net:',
          sum(p.numel() for p in fista_net.parameters()))

Total number of parameters fista net: 74599


In [9]:
dt = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
if START_EPOCH:
    dt = START_RUN
args = {
    'model_name': 'FISTANet',
    'num_epochs': EPOCH_NO,
    'lr': LEARNING_RATE,
    'data_dir': DATA_DIR,
    'save_path': f'./runs/{dt}',
    'start_epoch': START_EPOCH,
    'start_run': START_RUN,
    'multi_gpu': False,
    'device': device,
    'log_interval': LOG_INTERVAL,
    'test_epoch': TEST_EPOCH,
    'lr_dec_after': LR_DEC_AFTER,
    'lr_dec_every': LR_DEC_EVERY,
    'lambda_sp_loss': LAMBDA_SP_LOSS,
    'lambda_pred_sp_loss': LAMBDA_PRED_SP_LOSS,
    'lambda_sym_loss': LAMBDA_SYM_LOSS
}

In [10]:
solver = Solver(fista_net, Psi, trn_ldr, val_ldr, BATCH_SIZE, args, tst_ldr)

In [ ]:
solver.train()

Training epoch 1...

Train Epoch: 1 [0/8000 (0%)]	Batch Loss: 135937762.797572	Learning Rate (w_theta): 0.001000	 TIME:6.7s
				Disc: 135761888.851090		Sym: 175843.375000		Spars: 30.571482		Pred Spars: 0.000000
	 TVw: -0.501000 | TVb: -1.999000 | GSw: -0.201000 | GSb: 0.099000 | TSUw: 0.499000 | TSUb: 0.001000

Train Epoch: 1 [4000/8000 (50%)]	Batch Loss: 141908036.054484	Learning Rate (w_theta): 0.001000	 TIME:8.4s
				Disc: 141733877.067385		Sym: 174107.875000		Spars: 51.112099		Pred Spars: 0.000000
	 TVw: -0.504344 | TVb: -2.002264 | GSw: -0.204998 | GSb: 0.095002 | TSUw: 0.495002 | TSUb: 0.004998
Validating epoch 1...
-------------------------------------------
Epoch statistics:
Average training loss: 149515207.36051577
Average validation loss: 133115207.49378775
Training epoch 2...

Train Epoch: 2 [0/8000 (0%)]	Batch Loss: 145685645.471958	Learning Rate (w_theta): 0.001000	 TIME:10.6s
				Disc: 145503403.010898		Sym: 182134.765625		Spars: 107.695435		Pred Spars: 0.000000
	 TVw: -0